# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [4]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Yuxuan"
# Set the data and output directories
DATA_ROOT = '/home/yxcheng'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Celiac-Disease'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [5]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb']

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Breast_Cancer_(BRCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_BRCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [29]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [30]:
clinical_data_cols[:10]

['AJCC_Stage_nature2012',
 'Age_at_Initial_Pathologic_Diagnosis_nature2012',
 'CN_Clusters_nature2012',
 'Converted_Stage_nature2012',
 'Days_to_Date_of_Last_Contact_nature2012',
 'Days_to_date_of_Death_nature2012',
 'ER_Status_nature2012',
 'Gender_nature2012',
 'HER2_Final_Status_nature2012',
 'Integrated_Clusters_no_exp__nature2012']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

In [ ]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "Celiac-Disease"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE193442',
 'GSE113469',
 'GSE138297',
 'GSE106260',
 'GSE84745',
 'GSE23418',
 'GSE112102',
 'GSE87629',
 'GSE18123',
 'GSE72625',
 'GSE164883',
 'GSE84739']

Repeat the below steps for all the accession numbers

In [17]:
cohort = accession_num = "GSE112102"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/home/yxcheng/GEO/Celiac-Disease/GSE112102/GSE112102_family.soft.gz',
 '/home/yxcheng/GEO/Celiac-Disease/GSE112102/GSE112102_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [18]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"First Degree Relatives (FDRs) of Patients with Celiac Disease Harbour an Intestinal Transcriptomic Signature that Might Protect Them from Enterocyte Damage"
!Series_summary	"Patients with Celiac Disease, first degree relatives of celiac patients and control groups displayed significant differential gene expression."
!Series_overall_design	"All subjects were underwent upper GI endoscopic examination. Multiple mucosal biopsies were obtained using spike biopsy forceps. Four bits of tissues in 10% formal saline were sent for histological assessment for villous architecture assessment. Other 4-6 bites of mucosal biopsies were taken in RNA later for gene expression analysis. In this study, FDRs were selected randomly and none of the FDRs were related to Celiac patients."


In [19]:
clinical_data.head()

,!Sample_geo_accession,GSM3057759,GSM3057760,GSM3057761,GSM3057762,GSM3057763,GSM3057764,GSM3057765,GSM3057766,GSM3057767,...,GSM3057785,GSM3057786,GSM3057787,GSM3057788,GSM3057789,GSM3057790,GSM3057791,GSM3057792,GSM3057793,GSM3057794
0,!Sample_characteristics_ch1,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,...,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus,tissue: intestinal mucus
1,!Sample_characteristics_ch1,group: CeD,group: CeD,group: CeD,group: CeD,group: CeD,group: CeD,group: CeD,group: CeD,group: CeD,...,group: FDR,group: FDR,group: FDR,group: FDR,group: FDR,group: FDR,group: FDR,group: FDR,group: FDR,group: FDR
2,!Sample_characteristics_ch1,age: 40,age: 21,age: 23,age: 21,age: 23,age: 17,age: 19,age: 21,age: 18,...,age: 17,age: 42,age: 35,age: 32,age: 18,age: 42,age: 42,age: 21,age: 42,age: 42
3,!Sample_characteristics_ch1,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,...,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive,hla-dq2/dq8: positive
4,!Sample_characteristics_ch1,gender: Male,gender: Male,gender: Male,gender: Female,gender: Female,gender: Female,gender: Male,gender: Male,gender: Female,...,gender: Male,gender: Male,gender: Male,gender: Female,gender: Female,gender: Female,gender: Male,gender: Male,gender: Female,gender: Female


In [20]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: intestinal mucus'],
 1: ['group: CeD', 'group: control', 'group: FDR'],
 2: ['age: 40',
  'age: 21',
  'age: 23',
  'age: 17',
  'age: 19',
  'age: 18',
  'age: 20',
  'age: 32',
  'age: 38',
  'age: 42',
  'age: 27',
  'age: 28',
  'age: 26',
  'age: 25',
  'age: 35'],
 3: ['hla-dq2/dq8: positive', 'hla-dq2/dq8: negative'],
 4: ['gender: Male', 'gender: Female']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [21]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Celiac-Disease\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Celiac-Disease\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [27]:
is_gene_availabe = True
trait_row = 1
age_row= 2
gender_row = 4

trait_type = 'binary'

In [28]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [30]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'group: CeD':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: Female':
        return 0
    elif gender_string == 'gender: Male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, 23, 1)

In [31]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM3057759,GSM3057760,GSM3057761,GSM3057762,GSM3057763,GSM3057764,GSM3057765,GSM3057766,GSM3057767,GSM3057768,...,GSM3057785,GSM3057786,GSM3057787,GSM3057788,GSM3057789,GSM3057790,GSM3057791,GSM3057792,GSM3057793,GSM3057794
Celiac-Disease,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Age,40,21,23,21,23,17,19,21,18,20,...,17,42,35,32,18,42,42,21,42,42
Gender,1,1,1,0,0,0,1,1,0,1,...,1,1,1,0,0,0,1,1,0,0


### Genetic data preprocessing and final filtering

In [32]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM3057759,GSM3057760,GSM3057761,GSM3057762,GSM3057763,GSM3057764,GSM3057765,GSM3057766,GSM3057767,GSM3057768,...,GSM3057785,GSM3057786,GSM3057787,GSM3057788,GSM3057789,GSM3057790,GSM3057791,GSM3057792,GSM3057793,GSM3057794
ID,,,,,,,,,,,,,,,,,,,,,
ILMN_1343291,14.726150,14.789195,14.654076,14.789195,14.829983,14.677568,14.859861,14.859861,14.654076,14.573123,...,13.892160,13.962654,13.892160,13.991626,13.866719,13.877081,13.849746,13.788155,13.962654,13.611311
ILMN_1343295,12.111041,12.461130,12.959382,12.193488,12.688519,12.514982,12.481417,12.550157,12.133797,12.195513,...,11.242588,11.148588,11.312364,11.403161,11.345794,11.529301,11.312364,11.224584,11.242588,9.804207
ILMN_1651228,11.058577,10.995388,10.937940,10.714748,10.918051,10.706725,10.860812,10.734903,11.333399,11.459043,...,13.107793,12.769842,12.999748,12.875990,13.232136,13.135390,13.068756,13.012822,13.046905,12.502037
ILMN_1651229,6.125432,6.221028,6.762562,6.538092,6.415023,6.351378,6.494871,6.373543,6.719446,6.866318,...,6.985115,6.890756,7.049083,6.966247,7.086769,7.043384,7.018352,7.056027,7.005621,7.080838
ILMN_1651237,7.504694,7.663687,7.641847,7.199347,7.869448,8.048453,6.971181,7.562990,7.578632,8.105624,...,7.209995,7.128377,7.275932,6.789204,7.047561,7.108292,6.998213,6.942584,6.976867,6.726627


In [35]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['ILMN_1343291',
 'ILMN_1343295',
 'ILMN_1651228',
 'ILMN_1651229',
 'ILMN_1651237',
 'ILMN_1651254',
 'ILMN_1651262',
 'ILMN_1651278',
 'ILMN_1651282',
 'ILMN_1651285',
 'ILMN_1651315',
 'ILMN_1651336',
 'ILMN_1651346',
 'ILMN_1651347',
 'ILMN_1651364',
 'ILMN_1651378',
 'ILMN_1651385',
 'ILMN_1651405',
 'ILMN_1651415',
 'ILMN_1651429']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [36]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651228', 'ILMN_1651229', 'ILMN_1651237', 'ILMN_1651254', 'ILMN_1651262', 'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651315', 'ILMN_1651336', 'ILMN_1651346', 'ILMN_1651347', 'ILMN_1651364', 'ILMN_1651378', 'ILMN_1651385', 'ILMN_1651405', 'ILMN_1651415', 'ILMN_1651429']\n"

If not required, jump directly to the gene normalization step

In [37]:
requires_gene_mapping = True

In [39]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGAGGGCGCGTAGTTTGCA

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [40]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_geno

In [41]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [42]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

9 input query terms found dup hits:	[('ABCC6P1', 2), ('ABCC6P2', 3), ('ANXA2P1', 2), ('ANXA2P2', 2), ('ANXA2P3', 2), ('AQP7P1', 2), ('AQ
190 input query terms found no hit:	['2-Mar', '3-Mar', '5-Mar', '6-Mar', '7-Mar', 'A2LD1', 'AADACL1', 'AARS', 'ABP1', 'ACN9', 'ADCK4', '
3 input query terms found dup hits:	[('CCT6P1', 2), ('CECR7', 2), ('CELP', 2)]
350 input query terms found no hit:	['C16orf48', 'C16orf5', 'C16orf53', 'C16orf56', 'C16orf57', 'C16orf58', 'C16orf61', 'C16orf62', 'C16
2 input query terms found dup hits:	[('DHRS4L1', 3), ('DUXAP3', 2)]
163 input query terms found no hit:	['CP110', 'CPSF3L', 'CREBL1', 'CRIPAK', 'CRKRS', 'CROP', 'CRSP9', 'CSDA', 'CSNK2A1P', 'CSRP2BP', 'CT
7 input query terms found dup hits:	[('GTF2H2B', 2), ('GTF2IP1', 2), ('HCG4', 9), ('HLA-DRB6', 2), ('HLA-H', 9), ('HNRNPA3P1', 2), ('HSP
171 input query terms found no hit:	['FARSLB', 'FBXL10', 'FBXL11', 'FBXO18', 'FDX1L', 'FER1L3', 'FGFR1OP', 'FKBP9L', 'FKSG30', 'FLJ10081
5 input query terms found dup h

In [43]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [44]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 36 samples.


In [45]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Celiac-Disease', the least common label is '1.0' with 12 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Celiac-Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 21.0
  50% (Median): 27.0
  75%: 32.75
Min: 17.0
Max: 42.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 14 occurrences. This represents 38.89% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [46]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [47]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [48]:
from utils import *

In [49]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE112102,True,True,False,True,True,36,


In [50]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [51]:
merged_data.head()

,Celiac-Disease,Age,Gender,A1CF,A2M,AAAS,AACS,AADAC,AAGAB,AAMP,...,ZSWIM5,ZSWIM6,ZSWIM7,ZW10,ZWILCH,ZWINT,ZYG11B,ZYX,ZZEF1,ZZZ3
0,1.0,40.0,1.0,11.299358,9.969001,5.565968,7.428344,12.248039,7.320517,8.337521,...,7.493574,7.981288,7.432216,9.224346,8.270384,7.505519,8.994590,8.221540,9.104346,8.227451
1,1.0,21.0,1.0,9.462507,10.668462,5.422591,7.513717,11.232973,7.935180,8.041101,...,6.073549,8.102657,7.286231,9.426578,9.041429,8.342420,9.018129,8.485001,8.614329,8.515513
2,1.0,23.0,1.0,10.313678,10.118989,5.724798,7.138012,11.206594,8.725462,8.218250,...,6.412018,7.795335,7.241113,9.043638,8.938601,8.163404,8.803716,8.359158,8.971511,8.390954
3,1.0,21.0,0.0,10.973610,9.938324,5.009443,6.496277,11.869650,7.705516,8.111973,...,7.550811,8.447171,6.953007,8.883935,8.013349,7.383146,9.262534,7.924810,9.412874,8.333337
4,1.0,23.0,0.0,10.635063,10.064946,5.280715,7.002643,11.596136,8.022002,8.179128,...,7.133505,8.130783,6.914437,8.939898,8.716706,7.574235,8.950163,8.174673,9.388130,8.511889


In [52]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [53]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [54]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [55]:
if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [56]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 45.71% ± 24.58%


In [57]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 100.00% ± 0.00%


In [58]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [59]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.2123
Found 32 genes with non-zero coefficients associated with the trait 'Celiac-Disease' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
Variable  Coefficient  Absolute Coefficient
   TBX10    -4.236479              4.236479
    BRI3     3.626726              3.626726
    NOS2    -3.616521              3.616521
   SAP30     3.596229              3.596229
   PTRH2     3.436598              3.436598
   NUBP1     3.335510              3.335510
    LSM2     3.314344              3.314344
     DBI     3.210807              3.210807
   COPS6     3.063829              3.063829
   MTHFS     2.597924              2.597924
    GMNN     2.344937              2.344937
   ISG20    -2.245319              2.245319
     GIP     2.187603              2.187603
  GIMAP8    -2.008777              2.008777
  SLC2A3    -1.874839              1.874839
   RRP7A    -1.577064 

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [60]:

interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.2123
Found 0 significant genes associated with the trait 'Celiac-Disease' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
